In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyomo.environ import *

In [10]:
bus = ['Barra 1', 'Barra 2', 'Barra 3']
Nb = len(bus)

ger = ['Gerador_1', 'Gerador_2']
Ng = len(ger)
p_min = [0, 0]
p_max = [390, 9999]
C_ener = [10, 20]

loads = ['Carga_1']
Nc = len(loads)
dem = [450]

ger_loc = pd.DataFrame([[1,0,0], [0,1,0]], columns=bus, index=ger)
loads_loc = pd.DataFrame([[0,0,1]], columns=bus, index=loads)

conex = pd.DataFrame([[0,1,1],[1,0,1],[1,1,0]], columns=bus, index=bus) #Matriz de conexões
x_line = pd.DataFrame([[0,1,1],[1,0,1],[1,1,0]], columns=bus, index=bus) #Matriz de reatâncias
t_line = pd.DataFrame([[0,200,260],[200,0,200],[260,200,0]], columns=bus, index=bus) #Matriz de limites térmicos das linhas

In [11]:
model = ConcreteModel()

#Geradores Térmicos:
model.ger = Set(initialize=ger)
model.c_ener = Param(model.ger, initialize={ger[i]: C_ener[i] for i in range(Ng)})
model.pmax_ter = Param(model.ger, initialize={ger[i]: p_max[i] for i in range(Ng)})
model.pmin_ter = Param(model.ger, initialize={ger[i]: p_min[i] for i in range(Ng)})

#Cargas
model.loads = Set(initialize=loads)
model.dem = Param(model.loads, initialize={loads[i]: dem[i] for i in range(Nc)})

#Barras e Localizações
model.bus = Set(initialize=bus)
model.ger_loc = Param(model.ger, model.bus, initialize=lambda model, g, b: ger_loc.loc[g, b], default=0, mutable=False)
model.loads_loc = Param(model.loads, model.bus, initialize=lambda model, l, b: loads_loc.loc[l, b], default=0, mutable=False)
model.conex = Param(model.bus, model.bus, initialize=lambda model, b1, b2: conex.loc[b1, b2], default=0, mutable=False)
model.x_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: x_line.loc[b1, b2], default=0, mutable=False)
model.t_line = Param(model.bus, model.bus, initialize=lambda model, b1, b2: t_line.loc[b1, b2], default=0, mutable=False)

In [12]:
# Declarar Variáveis
model.P_ger = Var(model.ger, domain=NonNegativeReals) # Produção dos Geradores Térmicos
model.theta = Var(model.bus, domain=Reals)

In [13]:
#Restrições de Limites:
def prod_min(model, gerador):
    return model.P_ger[gerador] >= model.pmin_ter[gerador]
model.prod_min = Constraint(model.ger, rule=prod_min)

def prod_max(model, gerador):
    return model.P_ger[gerador] <= model.pmax_ter[gerador]
model.prod_max = Constraint(model.ger, rule=prod_max)

def flow_limit(model, barra1, barra2):
    if model.conex[barra1, barra2] == 1:
        return (model.theta[barra1] - model.theta[barra2])/model.x_line[barra1,barra2] <= model.t_line[barra1,barra2]
    else:
        return Constraint.Skip
model.flow_limit = Constraint(model.bus, model.bus, rule=flow_limit)

#Restrição de Balanço
def balanco(model, bus):
    thermal_generation = sum(model.P_ger[g]*model.ger_loc[g,bus] for g in model.ger)
    load_demand = sum(model.dem[l]*model.loads_loc[l,bus] for l in model.loads)
    line_flow = sum((model.theta[bus] - model.theta[bar])/model.x_line[bus, bar] for bar in model.bus if model.conex[bus,bar]==1)
    return thermal_generation - load_demand == line_flow
model.balanco = Constraint(model.bus, rule=balanco)

In [14]:
#Função Objetivo
def objective_rule(model):
    return sum(model.c_ener[i]*model.P_ger[i] for i in model.ger)
model.objective = Objective(rule=objective_rule, sense=minimize)

In [15]:
#Otimizando:
solver = SolverFactory('glpk')
resultados = solver.solve(model, tee=False)

In [16]:
#Relatório de Resultados
print('Status Final do Problema de Otimização:', resultados.solver.status, '\n')
print('Condição de Término:', resultados.solver.termination_condition, '\n')
print('Resultado Função Objetivo: R$', value(model.objective), '\n')

for gerador in model.ger:
    print('Energia gerada do {}: {:.2f} MWh'.format(gerador, model.P_ger[gerador].value))

for barra1 in model.bus:
    for barra2 in model.bus:
        if model.conex[barra1, barra2] == 1:
            flux = (model.theta[barra1].value - model.theta[barra2].value)/model.x_line[barra1, barra2]
            print(f'Fluxo de {barra1} para {barra2}: {flux}')

Status Final do Problema de Otimização: ok 

Condição de Término: optimal 

Resultado Função Objetivo: R$ 5700.0 

Energia gerada do Gerador_1: 330.00 MWh
Energia gerada do Gerador_2: 120.00 MWh
Fluxo de Barra 1 para Barra 2: 70.0
Fluxo de Barra 1 para Barra 3: 260.0
Fluxo de Barra 2 para Barra 1: -70.0
Fluxo de Barra 2 para Barra 3: 190.0
Fluxo de Barra 3 para Barra 1: -260.0
Fluxo de Barra 3 para Barra 2: -190.0
